In [1]:
from __future__ import print_function
import keras
import tensorflow as tf
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import LeakyReLU, BatchNormalization
from keras.optimizers import SGD
import os
import numpy as np

batch_size = 128
num_classes = 10
epochs = 50
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()

model.add(Conv2D(64, (3, 3),
                        input_shape=(32, 32, 3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dense(10))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)


# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Using real-time data augmentation.
Epoch 1/50
391/391 [==============================] - 80s 204ms/step - loss: 2.0359 - acc: 0.2467 - val_loss: 1.7093 - val_acc: 0.3863
Epoch 2/50
391/391 [==============================] - 26s 66ms/step - loss: 1.6714 - acc: 0.3874 - val_loss: 1.4193 - val_acc: 0.4803
Epoch 3/50
391/391 [==============================] - 26s 66ms/step - loss: 1.4897 - acc: 0.4537 - val_loss: 1.2910 - val_acc: 0.5310
Epoch 4/50
391/391 [==============================] - 26s 67ms/step - loss: 1.3677 - acc: 0.5036 - val_loss: 1.1605 - val_acc: 0.5826
Epoch 5/50
391/391 [==============================] - 26s 66ms/step - loss: 1.2554 - acc: 0.5493 - val_loss: 1.0897 - val_acc: 0.6106
Epoch 6/50
391/391 [==============================] - 26s 66ms/step - loss: 1.1597 - acc: 0.5868 - val_loss: 1.0017 - val_acc: 0.6474
Epoch 7/50
391/391 [==============================] - 26s 66ms/step - loss: 1.0785 - acc: 0.6195 - val_loss: 0.9169 - val_acc: 0.6794
Epoch 8/50
391/391 [======

In [2]:
from keras.backend import get_session
import os
if not os.path.exists('../tf/pretrained/'):
    os.makedirs('../tf/pretrained/')
sess = get_session()
saver = tf.train.Saver()
saver.save(sess,'../tf/pretrained/model')

'../tf/pretrained/model'